# セマンティックセグメンテーション用のDatasetとDataLoaderを作成

## Library 

In [20]:
import os.path as osp
from PIL import Image

import matplotlib.pyplot as plt
import numpy as np
import torch
import torch.utils.data as data

%matplotlib inline

## データのパスリストを作成

In [21]:
# これは関数で良い
def make_data_path(root_path):
    """
    学習用、検証用の画像データとアノテーションデータへのファイルパスリストを作成する
    
    Parameters
    -----------------
    root_path : str
        データディレクトリへのパス
    
    Returns
    -----------------
    train_img_list : list
    train_anno_list : 
    val_img_list : list
    val_anno_list : 
    
    """
    
    # 画像ファイルとアノテーションファイルへのパスのテンプレートを作成
    # formatできる形式にしておく
    # アノテーションデータはpng画像
    imgpath_tmp = osp.join(root_path, 'JPEGImages', '%s.jpg')
    annopath_tmp = osp.join(root_path, 'SegmentationClass', '%s.png')
    
    # 訓練と検証それぞれのファイルIDを取得
    train_id_names = osp.join(root_path + 'ImageSets/Segmentation/train.txt')
    val_id_names = osp.join(root_path + 'ImageSets/Segmentation/val.txt')
    
    # 訓練データの画像ファイルとアノテーションファイルへのパスのリストを作成
    train_img_list = []
    train_anno_list = []
    for line in open(train_id_names):
        file_id = line.strip()  # 空白と改行を削除
        img_path = (imgpath_tmp % file_id)  # 雛形にfile_idを入れる   画像のパス
        anno_path = (annopath_tmp % file_id)   # 雛形にfile_idを入れる　　アノテーションのパス
        train_img_list.append(img_path)
        train_anno_list.append(anno_path)      
    
    # 検証データの画像ファイルとアノテーションファイルへのパスのリストを作成
    val_img_list = []
    val_anno_list = []
    for line in open(val_id_names):
        file_id = line.strip()
        img_path = (imgpath_tmp % file_id)
        anno_path = (annopath_tmp % file_id)
        val_img_list.append(img_path)
        val_anno_list.append(anno_path)
        
    return train_img_list, train_anno_list, val_img_list, val_anno_list

In [22]:
# make_data_pathの動作確認
root_path = '../2_objectdetection/data/VOCdevkit/VOC2012/'
train_img_list, train_anno_list, val_img_list, val_anno_list = make_data_path(root_path)
print(train_img_list[0])
print(train_anno_list[0])

../2_objectdetection/data/VOCdevkit/VOC2012/JPEGImages/2007_000032.jpg
../2_objectdetection/data/VOCdevkit/VOC2012/SegmentationClass/2007_000032.png


## DataTransformを作成

In [31]:
from utils.data_augumentation import Compose, Scale, RandomRotation, RandomMirror, Resize, Normalize_Tensor

class DataTransform():
    """
    画像とアノテーションの前処理クラス
    訓練時と検証時で異なる動作をする
    訓練時はaugmentationする 

    """
    def __init__(self, input_size, color_mean, color_std):
        # 初期化の時点でtrainとvalのComposeの辞書作っておく
        # その際に input_size, color_mean, color_stdが必要になってくる
        self.data_transform = {
            'train' : Compose([
                Scale(scale=[0.5, 1.5]),  # 画像の拡大縮小
                RandomRotation(angle=[-10, 10]),  # 回転
                RandomMirror(),  # 1/2の確率で左右反転
                Resize(input_size),
                Normalize_Tensor(color_mean, color_std)  # 色情報の標準化とテンソル化
            ]),
            'val' : Compose([
                Resize(input_size),
                Normalize_Tensor(color_mean, color_std)
            ])
        }
    
    def __call__(self, phase, img, anno_img):
        """
        Parameters
        -----------------
        phase : str
            'train' or 'val'
            
        """
        return self.data_transform[phase](img, anno_img)
    

## Datasetの作成
opencvではなくPillowを使う

In [36]:
class VOCDataset(data.Dataset):
    """
    VOC2012のDatasetを作成するクラス
    PytorchのDatasetクラスを継承
    """
    
    def __init__(self, img_list, anno_list, phase, transform):
        self.img_list = img_list
        self.anno_list = anno_list
        self.phase = phase
        self.transform = transform
        
    def __len__(self):
        """
        画像の枚数を返す
        """
        return len(self.img_list)
    
    def __getitem__(self, index):
        """
        前処理をした画像のTensor形式のデータとアノテーションを取得
        """
        img, anno_img = self.pull_item(index)
        return img, anno_img
    
    def pull_item(self, index):
        """
        画像のTensor形式のデータ、アノテーションを取得する
        """
        # 画像読み込み
        img_file_path = self.img_list[index]
        img = Image.open(img_file_path)
        
        # アノテーション画像読み込み
        anno_file_path = self.anno_list[index]    # self.img_listにしててshapeがおかしかったのを修正
        anno_img = Image.open(anno_file_path)
        
        # 前処理の実施
        # ここでDataTransformの__call__が実行される
        img, anno_img = self.transform(self.phase, img, anno_img)
        
        return img, anno_img

In [37]:
# Datasetの動作確認

color_mean = (0.485, 0.456, 0.406)
color_std = (0.229, 0.224, 0.225)
input_size = 475

# Dataset作成
train_dataset = VOCDataset(train_img_list, train_anno_list,phase='train', transform=DataTransform(input_size, color_mean, color_std))
val_dataset = VOCDataset(val_img_list, val_anno_list,phase='val', transform=DataTransform(input_size, color_mean, color_std))

# データ取り出し例
print(train_dataset.__getitem__(0)[0].shape)  # img
print(train_dataset.__getitem__(0)[1].shape)  # anno_img
print(train_dataset.__getitem__(0))                   # Sequence

torch.Size([3, 475, 475])
torch.Size([475, 475])
(tensor([[[-2.1179, -2.1179, -2.1179,  ..., -2.1179, -2.1179, -2.1179],
         [-2.1179, -2.1179, -2.1179,  ..., -2.1179, -2.1179, -2.1179],
         [-2.1179, -2.1179, -2.1179,  ..., -2.1179, -2.1179, -2.1179],
         ...,
         [-2.1179, -2.1179, -2.1179,  ..., -2.1179, -2.1179, -2.1179],
         [-2.1179, -2.1179, -2.1179,  ..., -2.1179, -2.1179, -2.1179],
         [-2.1179, -2.1179, -2.1179,  ..., -2.1179, -2.1179, -2.1179]],

        [[-2.0357, -2.0357, -2.0357,  ..., -2.0357, -2.0357, -2.0357],
         [-2.0357, -2.0357, -2.0357,  ..., -2.0357, -2.0357, -2.0357],
         [-2.0357, -2.0357, -2.0357,  ..., -2.0357, -2.0357, -2.0357],
         ...,
         [-2.0357, -2.0357, -2.0357,  ..., -2.0357, -2.0357, -2.0357],
         [-2.0357, -2.0357, -2.0357,  ..., -2.0357, -2.0357, -2.0357],
         [-2.0357, -2.0357, -2.0357,  ..., -2.0357, -2.0357, -2.0357]],

        [[-1.8044, -1.8044, -1.8044,  ..., -1.8044, -1.8044, -1.80

In [38]:
# データ取り出し例
# __getitem__は[添字]でアクセスできるようにするための特殊メソッド
print(train_dataset[0][0].shape)  # img
print(train_dataset[0][1].shape)  # anno_img
print(train_dataset[0])                   # Sequence

torch.Size([3, 475, 475])
torch.Size([475, 475])
(tensor([[[-2.1179, -2.1179, -2.1179,  ..., -2.1179, -2.1179, -2.1179],
         [-2.1179, -2.1179, -2.1179,  ..., -2.1179, -2.1179, -2.1179],
         [-2.1179, -2.1179, -2.1179,  ..., -2.1179, -2.1179, -2.1179],
         ...,
         [-2.1179, -2.1179, -2.1179,  ..., -2.1179, -2.1179, -2.1179],
         [-2.1179, -2.1179, -2.1179,  ..., -2.1179, -2.1179, -2.1179],
         [-2.1179, -2.1179, -2.1179,  ..., -2.1179, -2.1179, -2.1179]],

        [[-2.0357, -2.0357, -2.0357,  ..., -2.0357, -2.0357, -2.0357],
         [-2.0357, -2.0357, -2.0357,  ..., -2.0357, -2.0357, -2.0357],
         [-2.0357, -2.0357, -2.0357,  ..., -2.0357, -2.0357, -2.0357],
         ...,
         [-2.0357, -2.0357, -2.0357,  ..., -2.0357, -2.0357, -2.0357],
         [-2.0357, -2.0357, -2.0357,  ..., -2.0357, -2.0357, -2.0357],
         [-2.0357, -2.0357, -2.0357,  ..., -2.0357, -2.0357, -2.0357]],

        [[-1.8044, -1.8044, -1.8044,  ..., -1.8044, -1.8044, -1.80

## DataLoaderの作成

In [39]:
batch_size = 8
train_dataloader = data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_dataloader = data.DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

# 辞書で管理
dataloaders_dict = {'train' : train_dataloader, 'val' : val_dataloader}

In [40]:
# DataLoaderの動作確認
batch_iterator = iter(dataloaders_dict['val'])  # イテレータに変換
imgs, anno_imgs = next(batch_iterator)
print(imgs.shape)
print(anno_imgs.shape)

torch.Size([8, 3, 475, 475])
torch.Size([8, 475, 475])
